# An example workflow of the QAOA package

This notebook walks through a simple example workflow, from start to finish.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Type, Iterable

# import the neccesary pyquil modules
from qaoa.cost_function import QAOACostFunctionOnQVM, QAOACostFunctionOnWFSim
from pyquil.api import local_qvm, WavefunctionSimulator
from pyquil.paulis import PauliSum, PauliTerm

# import the QAOAParameters that we want to demo
from qaoa.parameters import AdiabaticTimestepsQAOAParameters,\
AlternatingOperatorsQAOAParameters, AbstractQAOAParameters, GeneralQAOAParameters,\
QAOAParameterIterator, FourierQAOAParameters

from vqe.optimizer import scipy_optimizer

## Set up the problem hyperparameters

We start by setting up the Hamiltonian encoding the cost function we wish to minimise, as well as the timesteps to run QAOA. We do so by first using the `AbstractQAOAParameterClass`, however we could just as easily set up any specific parametrisation directly - see the notebook _"A demo how to work with `QAOAParameter` classes"_ for more details. [Should hyperlink to that notebook when online]
 

In [2]:
hamiltonian = PauliSum.from_compact_str("0.7*Z0*Z1 + 1.2*Z0*Z2  + (-0.5)*Z0")
print("hamiltonian =", hamiltonian)

timesteps = 2
myAbstractParams = AbstractQAOAParameters([hamiltonian,timesteps])
print(myAbstractParams)

hamiltonian = (0.7+0j)*Z0*Z1 + (1.2+0j)*Z0*Z2
Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: []
	single_qubit_coeffs: []
	qubits_pairs: [[0, 1], [0, 2]]
	pair_qubit_coeffs: [0.7 1.2]
	timesteps: 2



## Specfify the variable parameters and create corresponding `QAOAParameters` object

Here we will set up `AlternatingOperatorsQAOAParameters` - this corresponds to the parametrisation of the original QAOA paper by Farhi et al, where the mixer and cost Hamiltonian each have one angle per timestep (giving a total of 2p parameters to optimise over).

In [3]:
# Specify some angles
betas          = [0.1, 0.6]
gammas_singles = [0.4, 0.5]
gammas_pairs   = [0.1, 0.3]
parameters = (betas, gammas_singles, gammas_pairs)

myAlternatingParams = AlternatingOperatorsQAOAParameters.from_AbstractParameters(myAbstractParams,parameters)
print(myAlternatingParams)

Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: []
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [0.1 0.6]
	gammas_singles: [0.4 0.5]
	gammas_pairs: [0.1 0.3]



## Set up the cost function and running the optimiser

Let's work on the wavefunction simulator (note this requires opening the qvm and quil compiler in separate terminal windows as explained in the Forest documentation). We create the `cost_function` object, with all the necessary inputs [Make another demo of the `cost_functions` or expand the details here]. 

[NOTES: (1) look into the `noisy` attribute here - has it been properly tested and benchmarked? (2) demo the use of the `log` attribute]

In [4]:
# Set up the WavefunctionSimulator
sim = WavefunctionSimulator()

In [5]:
cost_function = QAOACostFunctionOnWFSim(hamiltonian,
                                        params=myAlternatingParams,
                                        sim=sim,
                                        return_standard_deviation=True,
                                        noisy=False,
                                        log=[])
res = scipy_optimizer(cost_function, myAlternatingParams.raw(), epsilon=1e-3,
                      maxiter=500)
res


QVMError: The value of QVM::SIZE is 0, which is not The SIZE wasn't a multiple of 8..

The QVM returned the above error. This could be due to a bug in the server or a
bug in your code. If you suspect this to be a bug in pyQuil or Rigetti Forest,
then please describe the problem in a GitHub issue at:
    https://github.com/rigetti/pyquil/issues

We can check this against the value we should get, by finding the lowest eigenvalue of `hamiltonian`.

In [6]:
ham_matrix = hamiltonian.matrix()
eigs = np.linalg.eigvals(ham_matrix)
min(eigs)

(-2.4+0j)

Clearly we have not found the ground state energy. We can try increasing the number of timesteps:

In [7]:
betas          = [0.1, 0.6, 0.8]
gammas_singles = [0.4, 0.5, 0.6]
gammas_pairs   = [0.1, 0.3, 0.5]
parameters = (betas, gammas_singles, gammas_pairs)

myAlternatingParams_p3 = AlternatingOperatorsQAOAParameters([hamiltonian,3],parameters)

In [8]:
sim = WavefunctionSimulator()

cost_function = QAOACostFunctionOnWFSim(hamiltonian,
                                        params=myAlternatingParams_p3,
                                        sim=sim,
                                        return_standard_deviation=True,
                                        noisy=False,
                                        log=[])
res = scipy_optimizer(cost_function, myAlternatingParams_p3.raw(), epsilon=1e-3,
                      maxiter=500)
res

     fun: -2.003521859659225
   maxcv: 0.0
 message: 'Maximum number of function evaluations has been exceeded.'
    nfev: 500
  status: 2
 success: False
       x: array([-1.11708954,  0.7255448 ,  1.9007215 ,  0.94227126,  0.46208833,
        0.09262068, -0.29573531,  0.47288961,  0.93348071])

This is better, but still not the actual ground state. Since we don't seem to be doing very well with `AlternatingOperatorsQAOAParameters`, let's try the `GeneralQAOAParameters` parametrisation instead.

In [9]:
myGeneralParams_p3 = GeneralQAOAParameters.linear_ramp_from_hamiltonian(hamiltonian,timesteps=3)

cost_function = QAOACostFunctionOnWFSim(hamiltonian,
                                        params=myGeneralParams_p3,
                                        sim=sim,
                                        return_standard_deviation=True,
                                        noisy=False,
                                        log=[])
res = scipy_optimizer(cost_function, myGeneralParams_p3.raw(), epsilon=1e-3,
                      maxiter=500)
res

     fun: -2.3999737306057503
   maxcv: 0.0
 message: 'Maximum number of function evaluations has been exceeded.'
    nfev: 500
  status: 2
 success: False
       x: array([ 7.85109896e-01,  9.49886941e-01,  1.72715008e-01, -1.72675901e-04,
        5.12838922e-01,  8.08470075e-01,  4.05111852e-04,  2.72540223e-01,
        3.19109481e-01,  1.56088201e+00,  8.15131099e-01,  1.07833412e+00,
       -7.95895587e-05, -3.43042345e-05,  1.11732608e+00,  3.88988687e-01,
        9.67772564e-03,  6.80313612e-01])

Bingo!